# Train a Simplicial Convolutional Neural Network (SCNN)

In this notebook, we will create and train a convolutional neural network in the simplicial complex domain, as proposed in the paper by [Yang et. al : SIMPLICIAL CONVOLUTIONAL NEURAL NETWORKS (2022)](https://arxiv.org/pdf/2110.02585.pdf). 

### We train the model to perform:
    1.  Complex classification using the shrec16 benchmark dataset.
    2.  Node classification using the karate dataset 


## Simplicial Convolutional Neural Networks <a href="https://arxiv.org/pdf/2110.02585.pdf">[SCNN]</a>

At layer $t$, given the input simplicial (edge) feature matrix $\mathbf{H}_t$, the SCNN layer is defined as 
$$
    \mathbf{H}_{t+1} = \sigma \Bigg[ \mathbf{H}_t\mathbf{\Theta}_t + \sum_{p_d=1}^{P_d}(\mathbf{L}_{\downarrow,1})^{p_d}\mathbf{H}_t\mathbf{\Theta}_{t,p_d} + \sum_{p_u=1}^{P_u}(\mathbf{L}_{\uparrow,1})^{p_u}\mathbf{H}_{t}\mathbf{\Theta}_{t,p_u} \Bigg]
$$
where $p_d$ and $p_u$ are the lower and upper convolution orders, respectively, and $\mathbf{\Theta}_{t,p_d}$ and $\mathbf{\Theta}_{t,p_u}$ are the learnable weights.
One can use $(\mathbf{L}_{\uparrow,1})^{p_u}$ and $(\mathbf{L}_{\uparrow,1})^{p_d}$ to perform higher-order upper and lower convolutions.


To align with the notations in [Papillon et al : Architectures of Topological Deep Learning: A Survey of Topological Neural Networks (2023)](https://arxiv.org/abs/2304.10031), we can use the following to denote the above layer definition. 

🟥 $\quad m_{y \rightarrow \{z\} \rightarrow x}^{p_u(1 \rightarrow 2 \rightarrow 1)}  = ((L_{\uparrow,1})^{p_u})_{xy} \cdot h_y^{t,(1)} \cdot \theta^{t, p_u} $  -------- Aggregate from $p_u$-hop upper neighbor $y$ to $x$

🟥 $\quad m_{y \rightarrow \{z\} \rightarrow x}^{p_d(1 \rightarrow 0 \rightarrow 1)} = ((L_{\downarrow,1})^{p_d})_{xy} \cdot h_y^{t,(1)} \cdot \theta^{t, p_d} $ -------- Aggregate from $p_d$-hop lower neighbor $y$ to $x$

🟥 $\quad m^{(1 \rightarrow 1)}_{x \rightarrow x} = \theta^t \cdot h_x^{t, (1)}$ --------  Aggregate from $x$ itself

🟧 $\quad m_{x}^{p_u,(1 \rightarrow 2 \rightarrow 1)}  = \sum_{y \in \mathcal{L}_\uparrow(x)}m_{y \rightarrow \{z\} \rightarrow x}^{p_u,(1 \rightarrow 2 \rightarrow 1)}$  -------- Collect the aggregated information from the upper neighborhood

🟧 $\quad m_{x}^{p_d,(1 \rightarrow 0 \rightarrow 1)} = \sum_{y \in \mathcal{L}_\downarrow(x)}m_{y \rightarrow \{z\} \rightarrow x}^{p_d,(1 \rightarrow 0 \rightarrow 1)}$ -------- Collect the aggregated information from the lower neighborhood

🟧 $\quad m^{(1 \rightarrow 1)}_{x} = m^{(1 \rightarrow 1)}_{x \rightarrow x}$

🟩 $\quad m_x^{(1)}  = m_x^{(1 \rightarrow 1)} + \sum_{p_u=1}^{P_u} m_{x}^{p_u,(1 \rightarrow 2 \rightarrow 1)} + \sum_{p_d=1}^{P_d} m_{x}^{p_d,(1 \rightarrow 0 \rightarrow 1)}$ -------- Collect all the aggregated information 

🟦 $\quad h_x^{t+1, (1)} = \sigma(m_x^{(1)})$ -------- Pass through the nonlinearity



# 1. Comples Classification

In [39]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

import toponetx.datasets as datasets

from topomodelx.nn.simplicial.scnn_layer import SCNNLayer

# Pre-processing

## Import shrec dataset ##

We must first lift our graph dataset into the simplicial complex domain.

In [40]:
shrec, _ = datasets.mesh.shrec_16(size="small")

shrec = {key: np.array(value) for key, value in shrec.items()}

x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
x_2s = shrec["face_feat"]

ys = shrec["label"]
simplexes = shrec["complexes"]

Loading shrec 16 small dataset...

done!


## Consider using edge features for classification 

In [41]:
in_channels_0 = x_0s[-1].shape[1]
in_channels_1 = x_1s[-1].shape[1]
in_channels_2 = x_2s[-1].shape[1]

### Define Neighborhood Strctures
Get incidence matrices and Hodge Laplacians

In [42]:
max_rank = 2  # the order of the SC is two
incidence_1_list = []
incidence_2_list = []

laplacian_0_list = []
laplacian_down_1_list = []
laplacian_up_1_list = []
laplacian_2_list = []

for simplex in simplexes:
    incidence_1 = simplex.incidence_matrix(rank=1)
    incidence_2 = simplex.incidence_matrix(rank=2)
    laplacian_0 = simplex.hodge_laplacian_matrix(rank=0, weight=True)
    laplacian_down_1 = simplex.down_laplacian_matrix(rank=1, weight=True)
    laplacian_up_1 = simplex.up_laplacian_matrix(rank=1, weight=True)
    laplacian_2 = simplex.hodge_laplacian_matrix(rank=2, weight=True)

    incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
    incidence_2 = torch.from_numpy(incidence_2.todense()).to_sparse()
    laplacian_0 = torch.from_numpy(laplacian_0.todense()).to_sparse()
    laplacian_down_1 = torch.from_numpy(laplacian_down_1.todense()).to_sparse()
    laplacian_up_1 = torch.from_numpy(laplacian_up_1.todense()).to_sparse()
    laplacian_2 = torch.from_numpy(laplacian_2.todense()).to_sparse()

    incidence_1_list.append(incidence_1)
    incidence_2_list.append(incidence_2)
    laplacian_0_list.append(laplacian_0)
    laplacian_down_1_list.append(laplacian_down_1)
    laplacian_up_1_list.append(laplacian_up_1)
    laplacian_2_list.append(laplacian_2)

# Create the SCNN

In [43]:
from topomodelx.nn.simplicial.scnn_layer import SCNNLayer


class SCNN(torch.nn.Module):
    """Simplicial convolutional neural network implementation for complex classification.

    Note: At the last layer, we obtain the output on simplcies, e.g., edges.
    To perform the complex classification task for this challenge, we consider pass the final output to a linear layer and compute the average.

    Parameters
    ----------
    in_channels: int
        Dimension of input features
    intermediate_channels: int
        Dimension of features of intermediate layers
    out_channels: int
        Dimension of output features
    conv_order_down: int
        Order of lower convolution
    conv_order_up: int
        Order of upper convolution
    n_layers: int
        Numer of layers
    """

    def __init__(
        self,
        in_channels,
        intermediate_channels,
        out_channels,
        conv_order_down,
        conv_order_up,
        aggr_norm=False,
        update_func=None,
        n_layers=2,
    ):
        super().__init__()
        # First layer -- initial layer has the in_channels as input, and inter_channels as the output
        layers = [
            SCNNLayer(
                in_channels=in_channels,
                out_channels=intermediate_channels,
                conv_order_down=conv_order_down,
                conv_order_up=conv_order_up,
            )
        ]

        for _ in range(n_layers - 1):
            layers.append(
                SCNNLayer(
                    in_channels=intermediate_channels,
                    out_channels=out_channels,
                    conv_order_down=conv_order_down,
                    conv_order_up=conv_order_up,
                    aggr_norm=aggr_norm,
                    update_func=update_func,
                )
            )

        self.linear = torch.nn.Linear(out_channels, 1)
        self.layers = torch.nn.ModuleList(layers)

    def forward(self, x, laplacian_down, laplacian_up):
        """Forward computation.

        Parameters
        ---------
        x: tensor
            shape = [n_simplices, channels]
            node/edge/face features

        laplacian: tensor
            shape = [n_simplices,n_simplices]
            For node features, laplacian_down = None

        incidence_1: tensor
            order 1 incidence matrix
            shape = [n_edges, n_nodes]
        """
        for layer in self.layers:
            x = layer(x, laplacian_down, laplacian_up)

        x_1 = self.linear(x)
        one_dimensional_cells_mean = torch.nanmean(x_1, dim=0)
        one_dimensional_cells_mean[torch.isnan(one_dimensional_cells_mean)] = 0

        return one_dimensional_cells_mean

# Train the Neural Network

We specify the model with our pre-made neighborhood structures and specify an optimizer.

In [44]:
rank = 1  # simplex level
conv_order_down = 2
conv_order_up = 2
intermediate_channels = 4
out_channels = intermediate_channels
num_layers = 2

# select the simplex level
if rank == 0:
    laplacian_down = None
    laplacian_up = laplacian_0_list  # the graph laplacian
    conv_order_down = 0
    x = x_0s
    in_channels = in_channels_0
elif rank == 1:
    laplacian_down = laplacian_down_1_list
    laplacian_up = laplacian_up_1_list
    x = x_1s
    in_channels = in_channels_1
elif rank == 2:
    laplacian_down = laplacian_2_list
    laplacian_up = None
    x = x_2s
    in_channels = in_channels_2
else:
    raise ValueError(f"Rank must be not larger than 2 on this dataset")

In [45]:
model = SCNN(
    in_channels=in_channels,
    intermediate_channels=intermediate_channels,
    out_channels=out_channels,
    conv_order_down=conv_order_down,
    conv_order_up=conv_order_up,
    n_layers=num_layers,
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

In [46]:
test_size = 0.2
x_train, x_test = train_test_split(x, test_size=test_size, shuffle=False)

laplacian_down_train, laplacian_down_test = train_test_split(
    laplacian_down, test_size=test_size, shuffle=False
)
laplacian_up_train, laplacian_up_test = train_test_split(
    laplacian_up, test_size=test_size, shuffle=False
)
y_train, y_test = train_test_split(ys, test_size=test_size, shuffle=False)

In [47]:
test_interval = 1
num_epochs = 5

# select which feature to use for labeling
simplex_order_select = 1

for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    for x, laplacian_down, laplacian_up, y in zip(
        x_train, laplacian_down_train, laplacian_up_train, y_train
    ):
        x = torch.tensor(x, dtype=torch.float)
        y = torch.tensor(y, dtype=torch.float)
        optimizer.zero_grad()

        y_hat = model(x, laplacian_down, laplacian_up)

        # print(y_hat.shape)
        loss = loss_fn(y_hat, y)

        epoch_loss.append(loss.item())
        loss.backward()
        optimizer.step()

    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            for x, laplacian_down, laplacian_up, y in zip(
                x_test, laplacian_down_test, laplacian_up_test, y_test
            ):
                x = torch.tensor(x, dtype=torch.float)
                y = torch.tensor(y, dtype=torch.float)
                optimizer.zero_grad()

                y_hat = model(x, laplacian_down, laplacian_up)

                loss = loss_fn(y_hat, y)
            print(f"Test_loss: {loss:.4f}", flush=True)

Epoch: 1 loss: 1347.8446
Test_loss: 123.1380
Epoch: 2 loss: 411.3039
Test_loss: 40.4423
Epoch: 3 loss: 180.6270
Test_loss: 94.9483
Epoch: 4 loss: 123.9201
Test_loss: 101.3076
Epoch: 5 loss: 105.6582
Test_loss: 101.2524


# 2. Node Classification 

In [48]:
import toponetx.datasets.graph as graph

## Pre-processing
### Import Karate dataset

In [49]:
dataset = graph.karate_club(complex_type="simplicial")
print(dataset)

# Maximal simplex order
max_rank = dataset.dim
print("maximal simple order:", max_rank)

Simplicial Complex with shape (34, 78, 45, 11, 2) and dimension 4
maximal simple order: 4


### Define Neighborhood Strctures
Get incidence matrices and Hodge Laplacians

In [50]:
incidence_1 = dataset.incidence_matrix(rank=1)
incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
incidence_2 = dataset.incidence_matrix(rank=2)
incidence_2 = torch.from_numpy(incidence_2.todense()).to_sparse()
print(f"The incidence matrix B1 has shape: {incidence_1.shape}.")
print(f"The incidence matrix B2 has shape: {incidence_2.shape}.")

The incidence matrix B1 has shape: torch.Size([34, 78]).
The incidence matrix B2 has shape: torch.Size([78, 45]).


### Weighted Hodge Laplacians
In the original paper, the weighted versions of the Hodge Laplacians are used. However, the current TOPONETX package does not provide this weighting feature yet. 

In [51]:
laplacian_0 = dataset.hodge_laplacian_matrix(rank=0, weight=True)
laplacian_down_1 = dataset.down_laplacian_matrix(rank=1, weight=True)
laplacian_up_1 = dataset.up_laplacian_matrix(rank=1, weight=True)
laplacian_down_2 = dataset.down_laplacian_matrix(rank=2, weight=True)
laplacian_up_2 = dataset.up_laplacian_matrix(rank=2, weight=True)

laplacian_0 = torch.from_numpy(laplacian_0.todense()).to_sparse()
laplacian_down_1 = torch.from_numpy(laplacian_down_1.todense()).to_sparse()
laplacian_up_1 = torch.from_numpy(laplacian_up_1.todense()).to_sparse()
laplacian_down_2 = torch.from_numpy(laplacian_down_2.todense()).to_sparse()
laplacian_up_2 = torch.from_numpy(laplacian_up_2.todense()).to_sparse()

### Import signals
#### Depending on the task, we can perform learning on any order of the simplices. Thus, the corresponding order of the input can be selected. 

For example, performing learning on the edges, we use the input on edges $\mathbf{x}_1$

In [52]:
x_0 = []
for _, v in dataset.get_simplex_attributes("node_feat").items():
    x_0.append(v)
x_0 = torch.tensor(np.stack(x_0))
channels_nodes = x_0.shape[-1]
x_1 = []
for k, v in dataset.get_simplex_attributes("edge_feat").items():
    x_1.append(v)
x_1 = np.stack(x_1)
chennel_edges = x_1.shape[-1]
x_2 = []
for k, v in dataset.get_simplex_attributes("face_feat").items():
    x_2.append(v)
x_2 = np.stack(x_2)
channel_faces = x_2.shape[-1]
print(f"There are {x_0.shape[0]} nodes with features of dimension {x_0.shape[1]}.")
print(f"There are {x_1.shape[0]} edges with features of dimension {x_1.shape[1]}.")
print(f"There are {x_2.shape[0]} faces with features of dimension {x_2.shape[1]}.")

There are 34 nodes with features of dimension 2.
There are 78 edges with features of dimension 2.
There are 45 faces with features of dimension 2.


Define a function to select the features on certain order of simplices 

In [53]:
"""A function to obtain features based on the input: rank
"""


def get_simplicial_features(dataset, rank):
    if rank == 0:
        which_feat = "node_feat"
    elif rank == 1:
        which_feat = "edge_feat"
    elif rank == 2:
        which_feat = "face_feat"
    else:
        raise ValueError(
            f"input dimension must be 0, 1 or 2, because features are supported on nodes, edges and faces"
        )

    x = []
    for _, v in dataset.get_simplex_attributes(which_feat).items():
        x.append(v)

    x = torch.tensor(np.stack(x))
    return x

### Define binary labels and Prepare the training-testing split

In [54]:
y = np.array(
    [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        0,
        1,
        1,
        1,
        1,
        0,
        0,
        1,
        1,
        0,
        1,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
)
y_true = np.zeros((34, 2))
y_true[:, 0] = y
y_true[:, 1] = 1 - y
y_test = y_true[-4:]
y_train = y_true[:30]

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

# Create the SCNN for node classification
Use the SCNNLayer classm we create a neural network with stacked layers. A final linear layer produces an output with shape $n_{\rm{nodes}}\times 2$, so we can compare with the binary labels

In [55]:
from topomodelx.nn.simplicial.scnn_layer import SCNNLayer


class SCNN(torch.nn.Module):
    """Simplicial convolutional neural network implementation for binary node classification.

    Note: At the last layer, we obtain the output on simplcies, e.g., edges.
    To perform the node classification task for this challenge, we consider a projection step via the incidence matrix B_1 which obtains the node labels from the edge output, which was also done in "T Mitchell Roddenberry, Nicholas Glaze and Santiago Segarra. Principled simplicial neural networks for trajectory prediction. International Conference on Machine Learning. 2021"

    Parameters
    ----------
    in_channels: int
        Dimension of input features
    intermediate_channels: int
        Dimension of features of intermediate layers
    out_channels: int
        Dimension of output features
    conv_order_down: int
        Order of lower convolution
    conv_order_up: int
        Order of upper convolution
    n_layers: int
        Numer of layers
    """

    def __init__(
        self,
        in_channels,
        intermediate_channels,
        out_channels,
        conv_order_down,
        conv_order_up,
        aggr_norm=False,
        update_func=None,
        n_layers=2,
    ):
        super().__init__()
        # First layer -- initial layer has the in_channels as input, and inter_channels as the output
        layers = [
            SCNNLayer(
                in_channels=in_channels,
                out_channels=intermediate_channels,
                conv_order_down=conv_order_down,
                conv_order_up=conv_order_up,
            )
        ]

        for _ in range(n_layers - 1):
            layers.append(
                SCNNLayer(
                    in_channels=intermediate_channels,
                    out_channels=out_channels,
                    conv_order_down=conv_order_down,
                    conv_order_up=conv_order_up,
                    aggr_norm=aggr_norm,
                    update_func=update_func,
                )
            )

        self.linear = torch.nn.Linear(out_channels, 2)
        self.layers = torch.nn.ModuleList(layers)

    def forward(self, x, laplacian_down, laplacian_up, incidence_1):
        """Forward computation.

        Parameters
        ---------
        x: tensor
            shape = [n_simplices, channels]
            node/edge/face features

        laplacian: tensor
            shape = [n_simplices,n_simplices]
            For node features, laplacian_down = None

        incidence_1: tensor
            order 1 incidence matrix
            shape = [n_edges, n_nodes]
        """
        for layer in self.layers:
            x = layer(x, laplacian_down, laplacian_up)
        """
        Project the output from edges to nodes 
        incidence_1 @ x
        """
        logits = self.linear(incidence_1 @ x)
        return torch.softmax(logits, dim=-1)

# Train the SCNN 

In [56]:
"""
Select the simplex order, i.e., on which level of simplices the learning will be performed 
"""
rank = 1  # simplex level
conv_order_down = 2
conv_order_up = 2
x = get_simplicial_features(dataset, rank)
channels_x = x.shape[-1]
if rank == 0:
    laplacian_down = None
    laplacian_up = laplacian_0  # the graph laplacian
    conv_order_down = 0
elif rank == 1:
    laplacian_down = laplacian_down_1
    laplacian_up = laplacian_up_1
elif rank == 2:
    laplacian_down = laplacian_down_2
    laplacian_up = laplacian_up_2
else:
    raise ValueError(f"Rank must be not larger than 2 on this dataset")

intermediate_channels = 16
out_channels = intermediate_channels

num_layers = 2
model = SCNN(
    in_channels=channels_nodes,
    intermediate_channels=intermediate_channels,
    out_channels=out_channels,
    conv_order_down=conv_order_down,
    conv_order_up=conv_order_up,
    n_layers=num_layers,
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)

SCNN(
  (linear): Linear(in_features=16, out_features=2, bias=True)
  (layers): ModuleList(
    (0-1): 2 x SCNNLayer()
  )
)


In [57]:
test_interval = 2
num_epochs = 5
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    optimizer.zero_grad()

    y_hat = model(x, laplacian_down, laplacian_up, incidence_1)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        y_hat[: len(y_train)].float(), y_train.float()
    )
    epoch_loss.append(loss.item())
    loss.backward()
    optimizer.step()

    y_pred = torch.where(y_hat > 0.5, torch.tensor(1), torch.tensor(0))
    accuracy = (y_pred[-len(y_train) :] == y_train).all(dim=1).float().mean().item()
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f} Train_acc: {accuracy:.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            y_hat_test = model(x, laplacian_down, laplacian_up, incidence_1)
            y_pred_test = torch.where(
                y_hat_test > 0.5, torch.tensor(1), torch.tensor(0)
            )
            test_accuracy = (
                torch.eq(y_pred_test[-len(y_test) :], y_test)
                .all(dim=1)
                .float()
                .mean()
                .item()
            )
            print(f"Test_acc: {test_accuracy:.4f}", flush=True)

Epoch: 1 loss: 0.7199 Train_acc: 0.5667
Epoch: 2 loss: 0.7196 Train_acc: 0.5667
Test_acc: 0.0000
Epoch: 3 loss: 0.7194 Train_acc: 0.5667
Epoch: 4 loss: 0.7192 Train_acc: 0.5667
Test_acc: 0.0000
Epoch: 5 loss: 0.7190 Train_acc: 0.5667
